![terrainbento logo](../../../../media/terrainbento_logo.png)

# Introduction to the terrainbento output writers.


## Overview
This tutorial shows example usage of the terrainbento output writers. For comprehensive information about all options and defaults, refer to the [documentation](http://terrainbento.readthedocs.io/en/latest/). 

## Prerequisites
This tutorial assumes you have at least skimmed the [terrainbento manuscript](https://www.geosci-model-dev.net/12/1267/2019/) and worked through the [Introduction to terrainbento](http://localhost:8888/notebooks/example_usage/Introduction_to_terrainbento.ipynb) tutorial.


### terrainbento output writers

By default terrainbento will save a [NetCDF](https://www.unidata.ucar.edu/software/netcdf/) file at equal intervals specified by the ``output_interval`` parameter. For many applications it may be sufficient to run a model, save output, and then open the stack of NetCDFs for postprocessing (we recommend [xarray](http://xarray.pydata.org/en/stable/) and [holoviews](http://holoviews.org) for opening, analyzing, and plotting stacks of NetCDFs in python). 

If in your application, you want to have additional output written at the ``output_interval`` intervals, you can pass any number of functions or classes that write this ouput to terrainbento. These functions and classes are called **output writers**. There is no limit to what you can do with the output writers. You could use them to make plots to turn into an animation, write diagnostic information about model run time, or calculate an objective function value over time. 

To make the output writers support as many possible use cases, terrainbento was designed to take any number of functions or classes as output writers. 

### Reference material
If you are not familiar with user defined python functions, consider reviewing [this tutorial](https://www.datacamp.com/community/tutorials/functions-python-tutorial#udf). If you are not familiar with user defined python classes, consider referring to [this tutorial](https://www.digitalocean.com/community/tutorials/how-to-construct-classes-and-define-objects-in-python-3).


### Example Usage

To begin, we will import the required python modules. 

In [ ]:
from terrainbento import Basic

import os
import numpy as np
import pandas as pd
np.random.seed(42)

import matplotlib
from landlab.plot.imshow import imshow_grid
import matplotlib.pyplot as plt
import holoviews as hv
hv.notebook_extension('matplotlib')
import imageio

#Ignore warnings 
import warnings
warnings.filterwarnings('ignore')

# Figure properties
%matplotlib inline
import panel as pn
pn.pane.Matplotlib.tight = True

We begin by creating an example user defined output writer function. When a terrainbento model runs the `OutputWriter` function, it will pass the model instance to the output writer. Thus the output writer function must be able to make all required calculations and plots based on the model instance. It is almost always the case that a user will want to write some sort of output -- typically a text file or plot -- each time the output writer is run. 

A user has the option of appending the output writer output to a file or using the model attributes `model.model_time` or `model.iteration` in the output file name to distinguish it from other files.  



In [ ]:
def write_avg_el(model):
    # mean elevation
    avg = model.z.mean()    
    fname = 'output/time_avg.txt'    
    if os.path.exists(fname) is False:
        with open(fname, 'w') as f:
            f.write('model_time,avg_el\n')

    with open(fname, 'a') as f:
        f.write(
            str(model.model_time) + ', ' + str(avg) + '\n')

Next we construct the parameter dictionary we will use to run the model. 

In [ ]:
basic_params = {
    # create the Clock.
    "clock": {
        "start": 0,
        "step": 1000,
        "stop": 2e5
    },

    # Create the Grid
    "grid": {
        "RasterModelGrid": [
            (25, 40),
            {
                "xy_spacing": 40
            },
            {
                "fields": {
                    "node": {
                        "topographic__elevation": {
                            "random": [{
                                "where": "CORE_NODE"
                            }]
                        }
                    }
                }
            },
        ]
    },

    # Set up Boundary Handlers
    "boundary_handlers": {
        "NotCoreNodeBaselevelHandler": {
            "modify_core_nodes": True,
            "lowering_rate": -0.0005
        }
    },
    # Parameters that control output.
    "output_interval": 1e4,
    "save_first_timestep": True,
    "output_prefix": "output/model_basic_output_intro_ow1",
    "fields": ["topographic__elevation"],

    # Parameters that control process and rates.
    "water_erodibility": 0.0001,
    "m_sp": 0.5,
    "n_sp": 1.0,
    "regolith_transport_parameter": 0.01,
}

Unline the boundary handlers described in [this tutorial](http://localhost:8888/notebooks/example_usage/introduction_to_boundary_conditions.ipynb), we pass the output writer to terrainbento as a keyword argument. 

After instantiating the model, we run it. 

In [ ]:
basic = Basic.from_dict(basic_params,
                        output_writers={"function": [write_avg_el]})
basic.run()

First we will make a plot of the topography. We have evolved some nice valleys and ridges. 

In [ ]:
imshow_grid(basic.grid, basic.z)

Now that we've run the model we will compile all of the output into a dataframe and plot it. 

Next we will read in the output we wrote to the file `output/time_avg.txt` using [pandas](https://pandas.pydata.org) and remove the files created by running the model. In your usage you probably won't want to remove these files, but we are doing it so that running the model won't create lots of files in your directory. 

In [ ]:
df = pd.read_csv('output/time_avg.txt')
ds = basic.to_xarray_dataset(time_unit='years', space_unit='meters')

We can look at what is inside of the dataframe. 

In [ ]:
df

In [ ]:
df.columns

This dataframe shows us the history of drainage density through time. Next we will plot it. 

In [ ]:
plt.figure()
plt.plot(df.model_time, df.avg_el)
plt.xlabel('Model Time [years]')
plt.ylabel('Mean elevation, m')
plt.show()

# Let's make a movie
First, we make an image for each output interval.

In [ ]:
from landlab import imshow_grid

filenames = []
curT = 0
for i in range(ds.topographic__elevation.shape[0]):
    curT+=basic.output_interval
    filename = "figures/temp_output."+str(i)+".png"
    imshow_grid(basic.grid, ds.topographic__elevation.values[i, :, :], cmap="viridis", limits=(0, 25), output=filename,
               plot_name=str(curT) + ' model years')
    filenames.append(filename)    


Finally we compile the images into a gif.

In [ ]:
with imageio.get_writer("figures/terrainbento_example.gif", mode="I") as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        os.remove(filename)

Delete data

In [ ]:
basic.remove_output_netcdfs()
os.remove('output/time_avg.txt')

## Next Steps

- [Welcome page](../Welcome_to_TerrainBento.ipynb)


- There are three additional introductory tutorials: 

    1) [Introduction terrainbento](Introduction_to_terrainbento.ipynb) 
    
    2) [Introduction to boundary conditions in terrainbento](introduction_to_boundary_conditions.ipynb)
    
    3) **This Notebook**: [Introduction to output writers in terrainbento](introduction_to_output_writers.ipynb). 
    
    
- Five examples of steady state behavior in coupled process models can be found in the following notebooks:

    1) [Basic](../coupled_process_elements/model_basic_steady_solution.ipynb) the simplest landscape evolution model in the terrainbento package.

    2) [BasicVm](../coupled_process_elements/model_basic_var_m_steady_solution.ipynb) which permits the drainage area exponent to change

    3) [BasicCh](../coupled_process_elements/model_basicCh_steady_solution.ipynb) which uses a non-linear hillslope erosion and transport law

    4) [BasicVs](../coupled_process_elements/model_basicVs_steady_solution.ipynb) which uses variable source area hydrology

    5) [BasisRt](../coupled_process_elements/model_basicRt_steady_solution.ipynb) which allows for two lithologies with different K values
    
    6) [RealDEM](../coupled_process_elements/model_basic_realDEM.ipynb) Run the basic terrainbento model with a real DEM as initial condition. 